In [ ]:
#!pip install psycopg2-binary #for Michelle

In [ ]:
import numpy as np
import pandas as pd
import os, glob
import sqlalchemy
import sqlite3
from sqlite3 import Error
import sklearn as sk
from sklearn.linear_model import LogisticRegression
from sklearn import svm, metrics
#from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import math
import gzip
import csv
import matplotlib.pyplot as plt
import psycopg2

# Connecting to mimic postgres database

In [ ]:
from sqlalchemy import create_engine

# Postgres username, password, database name
# @team - you'll need to change this according to what your Postgres info is under the Properties tab of your server in PgAdmin

# Michelle's info
#POSTGRES_ADDRESS = '127.0.0.1'
#POSTGRES_PORT= '5432'
#POSTGRES_USERNAME= 'postgres'
#POSTGRES_PASSWORD= 'password'
#POSTGRES_DBNAME = 'mimic'

# Paroma's info 
POSTGRES_ADDRESS = 'localhost'   #127.0.0.1
POSTGRES_PORT= '5432'
POSTGRES_USERNAME= 'MacUser'
POSTGRES_PASSWORD= ''
POSTGRES_DBNAME = 'mimic'

# putting in info
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME,password=POSTGRES_PASSWORD,ipaddress=POSTGRES_ADDRESS,port=POSTGRES_PORT,dbname=POSTGRES_DBNAME))

# Create the connection
cnx = create_engine(postgres_str)


In [ ]:
# path with all normalized data -chartevents, lab events, procedure events
#path = "C:/Users/mnigc/Documents/PCM/" # for Michelle
path = "/Users/MacUser/Desktop/Fall_2020_Semester/Classes/Precision_Care_Medicine/Un-normalized_matched_patient_id_Data"

# FOR CHARTEVENTS
# loading all files from path with chartevent name
cohort_files = glob.glob(os.path.join(path, "lab_events_*"))

# initializing data frame
cohort_df = []
# loading all files into data frame
for f in cohort_files:
    df = pd.read_csv(f)
    df['file'] = f.split('/')[-1]
    cohort_df.append(df)
    
# concatenating files into single data frame    
cohort = pd.concat(cohort_df, ignore_index=True)

C:\Users\mnigc\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
#  getting subject ids from cohort files
patient_ids = cohort['subject_id']

# get unique patient ids
unique_ids=np.unique(patient_ids)

# clear NaN
unique_ids=unique_ids[~np.isnan(unique_ids)]

len(unique_ids)

2596

# Feature extraction and cleaning by group

In [ ]:
# find all the item ids - 


In [ ]:
# Albumin
# 50862 (Albumin), 51069 (Albumin, Urine), 51070 (Albumin/Creatinine/Urine)

# find albumin ids from d_items 
albumin_id=pd.read_sql_query('''SELECT itemid,linksto FROM d_items WHERE label='albumin';''',cnx)
print(albumin_id)

# find albumin entries in linksto tables for cohort
albumin_chart=pd.read_sql_query('''SELECT subject_id, hadm_id, charttime, value, valuenum, warning 
FROM chartevents
WHERE itemid IN (3066);''',cnx)
albumin_input=pd.read_sql_query('''SELECT subject_id, hadm_id, charttime, amount FROM inputevents_cv WHERE itemid =45403''',cnx)


# check to see if these people are in cohort
albumin_chart=albumin_chart[albumin_chart['subject_id'].isin(unique_ids)]
albumin_input=albumin_input[albumin_input['subject_id'].isin(unique_ids)]


print(albumin_chart)
print(albumin_input)

   itemid         linksto
0    3066     chartevents
1   45403  inputevents_cv
   subject_id  hadm_id           charttime value valuenum warning
0        1046   113191 2168-10-21 23:00:00  None     None    None
Empty DataFrame
Columns: [subject_id, hadm_id, charttime, amount]
Index: []


In [ ]:
# Anion Gap (calculated from Na, K, Cl, HCO3)

In [ ]:
# Bilirubin
# Bilirubin (51464), Bilirubin Crystals (51465)

# find bilirubin ids from d_items 
bilirubin_id=pd.read_sql_query('''SELECT itemid,label FROM d_labitems WHERE label LIKE '%Bilirubin%' or label LIKE '%Bilirubin' or label LIKE or label ='Bilirubin';''',cnx)
print(bilirubin_id)

# find albumin entries in linksto tables for cohort
bilirubin_chart=pd.read_sql_query('''SELECT subject_id, hadm_id, charttime, value, valuenum, warning 
FROM chartevents
WHERE itemid IN (3066);''',cnx)
albumin_input=pd.read_sql_query('''SELECT subject_id, hadm_id, charttime, amount FROM inputevents_cv WHERE itemid =45403''',cnx)


# check to see if these people are in cohort
albumin_chart=albumin_chart[albumin_chart['subject_id'].isin(unique_ids)]
albumin_input=albumin_input[albumin_input['subject_id'].isin(unique_ids)]


TypeError: dict is not a sequence

In [ ]:
# Chloride
# 50806 (Chloride, Whole Blood), 50902 (Chloride), 51078 (Chloride, Urine)

In [ ]:
# Creatinine
# 50912 (Creatinine), 51067 (24 hr Creatinine), 51070 (Albumin), 
# 51073 (Amylase/Creatinine Ratio/Urine), 51080 (Creatinine Clearance),
# 51081 (Creatinine, Serum), 51082 (Creatinine, Urine), 51099 (Protein/Creatinine Ratio),
# 51106 (Urine Creatinine)

In [ ]:
# Diastolic BP

In [ ]:
# Epinephrine

In [ ]:
# Glasgow Coma Scale

In [ ]:
# Glucose
# 50809 (Glucose), 50931 (Glucose), 51084 (Glucose, Urine), 51478 (Glucose)
# 51529 (Estimated Actual Glucose)

In [ ]:
# Heart rate

In [ ]:
# Hemoglobin
# 50811 (Hemoglobin), 50852 (% Hemoglobin A1c), 50855 (Absolute Hemoglobin),
# 51212 (Fetal Hemoglobin), 51222 (Hemoglobin), 51223 (Hemoglobin A2),
# 51224 (Hemoglobin C), 51225 (Hemoglobin F), 
# 51285 (Reticulocyte, Cellular Hemoglobin)

In [ ]:
# Intubation

In [ ]:
# Lactate
# 50813 (Lactate), 50954 (Lactate Dehydrogenase (LD))

In [ ]:
# Mean BP

In [ ]:
# Mechanical ventilation

In [ ]:
# pH
# 50820 (pH), 51094 (pH), 51491 (pH)

In [ ]:
# Platelet
# 51240 (Large Platelets), 51264 (Platelet Clumps), 51265 (Platelet Count)
# 51266 (Platelet Smear)

In [ ]:
# Potassium
# 50822 (Potassium, Whole Blood), 50971 (Potassium), 
# 51097 (Potassium, Urine)

In [ ]:
# Respiration rate

In [ ]:
# Sodium
# 50824 (Sodium, Whole Blood), 50983 (Sodium), 51100 (Sodium, Urine)

In [ ]:
# SOFA Score ?? ? ?  - Still need to get feedback from mentors re: inclusion

In [ ]:
# SpO2

In [ ]:
# Systolic BP

In [ ]:
# Temperature

In [ ]:
# Urine output

In [ ]:
# White blood count